In [45]:
import sys
sys.path.append('..')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()

In [117]:
# lang = 'zh-cn' if gezi.in_notebook() else sys.argv[1]
lang = 'it' if gezi.in_notebook() else sys.argv[1]
lang_ = lang.split('-')[-1]
ic(lang, lang_)

In [46]:
from nltk.tokenize import sent_tokenize
from googletrans import Translator
import time
trans = Translator()
ic(trans.translate('照片').text)

[09/02/22 23:10:35] 2180798362.py:5 in <module>
                    trans.translate('照片').text: 'photo'


'photo'

In [99]:
def translate(text, lang='zh-cn', max_len=512):
  num_words = len(text.split())
  if num_words <= max_len:
    mid_text = trans.translate(text, dest=lang).text
    target_text = trans.translate(mid_text, dest='en').text
    time.sleep(0.5)
  else:
    mid_texts = []
    target_texts = []
    sents = sent_tokenize(text)    
    num_sents = len(sents)
    parts = int(num_words / max_len) + 1
    if num_sents > parts:
      sents = np.array_split(sents, parts)
      for i in range(len(sents)):
        sents[i] = ''.join(sents[i])
    for sent in sents:
      # print(sent)
      mid_text = trans.translate(sent, dest=lang).text
      # print(mid_text)
      target_text = trans.translate(mid_text, dest='en').text
      # print(target_text)
      time.sleep(0.5)
      mid_texts.append(mid_text)
      target_texts.append(target_text)
    mid_text = ''.join(mid_texts)
    target_text = ''.join(target_texts)
  return mid_text, target_text

In [48]:
trans_dict = {}
trans_dict2 = {}

In [102]:
def translate_(text, lang):
  if text not in trans_dict or text not in trans_dict2:
    try:
      mid_text, target_text = translate(text, lang)
      trans_dict[text] = mid_text
      trans_dict2[text] = target_text
    except Exception as e:
      # ic(e, len(text.split()))
      # bads.append(text)
      time.sleep(1)
      try:
        mid_text, target_text = translate(text, lang, 256)
        trans_dict[text] = mid_text
        trans_dict2[text] = target_text
      except Exception as e:
        time.sleep(1)
        try:
          mid_text, target_text = translate(text, lang, 128)
          trans_dict[text] = mid_text
          trans_dict2[text] = target_text
        except Exception as e:
          time.sleep(1)
          try:
            mid_text, target_text = translate(text, lang, 64)
            trans_dict[text] = mid_text
            trans_dict2[text] = target_text
          except Exception as e:
            ic(e, len(text.split()))

In [116]:
d = pd.read_csv(f'{FLAGS.root}/train.csv')

In [103]:
gezi.prun_loop(lambda x: translate_(x, lang), d.full_text.values, 1, desc='trans')

trans:   0%|          | 0/3911 [00:00<?, ?it/s]

In [ ]:
ic(len(trans_dict), len(d))
assert len(trans_dict) == len(d)

In [107]:
d[f'{lang_}_text'] = d.full_text.apply(lambda x: trans_dict[x])

In [109]:
d[f'trans_{lang_}'] = d.full_text.apply(lambda x: trans_dict2[x])

In [113]:
gezi.set_pd_widder()

In [114]:
d

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cn_text,trans_cn
0,0016926B079C,"I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortable at home.\n\nThe hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. Then you'll have to change. with the online classes you can wear anything and stay home and you wont need to stress about what to wear.\n\nmost students usually take showers before school. they either take it before they sleep or when they wake up. some students do both to smell good. that causes them do miss the bus and effects on there lesson time cause they come late to school. when u have online classes u wont need to miss lessons cause you can get everything set up and go take a shower and when u get out your ready to go.\n\nwhen your home your comfortable and you pay attention. it gives then an advantage to be smarter and even pass there classmates on class work. public schools are difficult even if you try. some teacher dont know how to teach it in then way that students understand it. that causes students to fail and they may repeat the class.",3.5,3.5,3.0,3.0,4.0,3.0,我认为学生将从家里学习中受益，因为他们不必在清晨淋浴并在那里做头发。只上课可以帮助他们，因为在那里，他们将受到更多关注。他们会在家里很舒服。\n\n学校最困难的部分正在做好准备。您醒来去刷牙，去壁橱，看着你的衣服。在您认为选择了一件衣服后，您可以去看镜子，要么不喜欢它，要么看污渍。然后，您必须更改。通过在线课程，您可以穿任何衣服并待在家里，您不需要强调穿什么衣服。\n\n大多数学生通常在放学前洗澡。他们要么在睡觉之前就服用它，要么醒来。有些学生俩都闻起来很香。这导致他们确实错过了公共汽车，并且对那里的影响时间，因为他们上学迟到了。当您有在线课程时，您将不需要错过课程，因为您可以设置所有东西，然后去洗个澡，当您准备好出去时。\n\n当您的房屋舒适并注意时。然后，它具有更聪明的优势，甚至通过班上的同学通过那里。即使您尝试，公立学校也很困难。一些老师不知道如何以学生理解它的方式教它。这会导致学生失败，他们可能会重复课程。,"I think students will benefit from learning at home because they don't have to shower in the morning and make their hair there. Only class can help them, because they will receive more attention there. They will be comfortable at home.\n\nThe most difficult part of the school is preparing. You wake up to brush your teeth, go to the closet, and look at your clothes. After you think you choose a dress, you can go to see the mirror, either you don't like it or look at the stains. Then you must change it. Through online courses, you can wear any clothes and stay at home. You don't need to emphasize what clothes you wear.\n\nMost students usually take a bath before school. They either take it before going to bed or wake up. Some students smell very fragrant. This caused them to miss the buses and affect the time there was because they were late in school. When you have an online course, you don't need to miss the course because you can set up everything and take a bath, when you are ready to go out.\n\nWhen your house is comfortable and pay attention. Then, it has a smarter advantage, and even through the classmates in the class. Even if you try, public schools are difficult. Some teachers do not know how to teach it in a way student understand it. This will cause students to fail, and they may repeat the curriculum."
1,0022683E9EA5,"When a problem is a change you have to let it do the best on you no matter what is happening it can change your mind. sometimes you need to wake up and look what is around you because problems are the best way to change what you want to change along time ago. A\n\nproblem is a change for you because it can make you see different and help you to understand how tings wok.\n\nFirst of all it can make you see different then the others. For example i remember that when i came to the United States i think that nothing was going to change me because i think that nothing was going to change me because everything was different that my country and then i realist that wrong because a problem may change you but sometimes

In [115]:
d.to_csv(f'{FLAGS.root}/train_{lang_}.csv', index=False)